## Ejercicio complementario 5

In [2]:
import warnings
warnings.filterwarnings('ignore')
from pyomo.environ import *

model = ConcreteModel()
model.Plantas = Set(initialize=['P1', 'P2'])
model.Municipios = Set(initialize=['Cucuta', 'Pamplona', 'Gramalote'])

oferta = {
    'P1': 150,
    'P2': 140
}

demanda = {
    'Cucuta': 80,
    'Pamplona': 50,
    'Gramalote': 55
}
costos = {
    ('P1', 'Cucuta'): 7,
    ('P1', 'Pamplona'): 2,
    ('P1', 'Gramalote'): 5,
    ('P2', 'Cucuta'): 6,
    ('P2', 'Pamplona'): 6,
    ('P2', 'Gramalote'): 3
}

model.oferta = Param(model.Plantas, initialize=oferta)
model.demanda = Param(model.Municipios, initialize=demanda)
model.costo = Param(model.Plantas, model.Municipios, initialize=costos)
model.x = Var(model.Plantas, model.Municipios, within=NonNegativeReals)

def funcion_objetivo(model):
    return sum(model.costo[p,m] * model.x[p,m]
               for p in model.Plantas
               for m in model.Municipios)

model.obj = Objective(rule=funcion_objetivo, sense=minimize)

def restriccion_oferta(model, p):
    return sum(model.x[p,m] for m in model.Municipios) <= model.oferta[p]

model.res_oferta = Constraint(model.Plantas, rule=restriccion_oferta)

def restriccion_demanda(model, m):
    return sum(model.x[p,m] for p in model.Plantas) >= model.demanda[m]

model.res_demanda = Constraint(model.Municipios, rule=restriccion_demanda)

solver = SolverFactory('appsi_highs')  
resultado = solver.solve(model, tee=True)


print("RESULTADOS ÓPTIMOS ")

for p in model.Plantas:
    for m in model.Municipios:
        if value(model.x[p,m]) > 0:
            print(f"Energía enviada de {p} a {m}: {value(model.x[p,m])} millones de kW")

print(f"\nCosto total mínimo: ${value(model.obj)}")


RESULTADOS ÓPTIMOS 
Energía enviada de P1 a Pamplona: 50.0 millones de kW
Energía enviada de P2 a Cucuta: 80.0 millones de kW
Energía enviada de P2 a Gramalote: 55.0 millones de kW

Costo total mínimo: $745.0


--- 
## Ejercicio complementario 6

In [3]:

model = ConcreteModel()

model.PRODUCTOS = Set(initialize=['Galletas', 'Cupcakes', 'Minitortas'])

utilidad = {
    'Galletas': 600,
    'Cupcakes': 4000,
    'Minitortas': 6000
}

demanda_min = {
    'Galletas': 40,
    'Cupcakes': 0,
    'Minitortas': 0
}

demanda_max = {
    'Galletas': None,
    'Cupcakes': 30,
    'Minitortas': 20
}

model.utilidad = Param(model.PRODUCTOS, initialize=utilidad)
model.demanda_min = Param(model.PRODUCTOS, initialize=demanda_min, default=0)
model.demanda_max = Param(model.PRODUCTOS, initialize=demanda_max, default=None)

model.x = Var(model.PRODUCTOS, within=NonNegativeIntegers)

def objetivo(model):
    return sum(model.utilidad[p] * model.x[p] for p in model.PRODUCTOS)

model.obj = Objective(rule=objetivo, sense=maximize)

def restriccion_total(model):
    return sum(model.x[p] for p in model.PRODUCTOS) <= 100

model.res_total = Constraint(rule=restriccion_total)

def restriccion_min(model, p):
    return model.x[p] >= model.demanda_min[p]

model.res_min = Constraint(model.PRODUCTOS, rule=restriccion_min)

def restriccion_max(model, p):
    if model.demanda_max[p] is None:
        return Constraint.Skip
    return model.x[p] <= model.demanda_max[p]

model.res_max = Constraint(model.PRODUCTOS, rule=restriccion_max)

solver = SolverFactory('appsi_highs')
solver.solve(model)

for p in model.PRODUCTOS:
    print(p, value(model.x[p]))
print("Utilidad total:", value(model.obj))


Galletas 50.0
Cupcakes 30.0
Minitortas 20.0
Utilidad total: 270000.0
